# XGBoost

XGBoost(eXtreme Gradient Boosting)は、分布型勾配ブースティングフレームワークで、Tianqi Chenにより開発されました。このフレームワークは高い効率と拡張性を持ち、欠損値の処理、正則化、並列処理などの特徴を持っています。さらに、決定木アルゴリズムをベースとしており、大規模なデータセットに対しても高速に動作し、精度も高いことで知られています。

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/tabledata/binary_classification/XGBoost.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install xgboost

In [4]:
import xgboost as xgb

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_breast_cancer

### データの用意

In [5]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [7]:
x = df.drop('target', axis = 1)
y = df['target']

X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

### XGBoostの学習

In [13]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_valid, label=y_valid)

# パラメータを設定
param = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'n_estimators': 10
}

# モデルを訓練
clf = xgb.train(param, dtrain, num_boost_round=10, evals=[(dtrain, 'train'), (dtest, 'eval')])


[0]	train-logloss:0.44242	eval-logloss:0.46755
[1]	train-logloss:0.31590	eval-logloss:0.35439
[2]	train-logloss:0.23584	eval-logloss:0.28176
[3]	train-logloss:0.18049	eval-logloss:0.24011
[4]	train-logloss:0.14210	eval-logloss:0.20182
[5]	train-logloss:0.11071	eval-logloss:0.17846
[6]	train-logloss:0.08864	eval-logloss:0.16165
[7]	train-logloss:0.07298	eval-logloss:0.15280
[8]	train-logloss:0.06182	eval-logloss:0.14094
[9]	train-logloss:0.05160	eval-logloss:0.13805


/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/Users/tanakatouma/vscode/data-science-wiki/.venv/lib/python3.9/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed 

### 予測

In [14]:
y_pred = clf.predict(dvalid)

In [15]:
auc = roc_auc_score(y_valid, y_pred)
print(f'AUC: {auc}')

AUC: 0.9911562397641664
